In [1]:
import tensorflow as tf

In [2]:
tf.reset_default_graph()

In [19]:
batch_size = 10
learning_rate = 0.01
training_epochs = 50
logs_path = "/tmp/mnist/2"

In [4]:
from tensorflow.examples.tutorials.mnist import input_data

In [5]:
mnist = input_data.read_data_sets('MNIST_data', one_hot=True)

Extracting MNIST_data/train-images-idx3-ubyte.gz
Extracting MNIST_data/train-labels-idx1-ubyte.gz
Extracting MNIST_data/t10k-images-idx3-ubyte.gz
Extracting MNIST_data/t10k-labels-idx1-ubyte.gz


In [6]:
with tf.name_scope('input'):
    x = tf.placeholder(tf.float32, shape=[None, 784], name='x-input')
    y_ = tf.placeholder(tf.float32, shape=[None, 10], name='y-input')

In [7]:
with tf.name_scope('weights'):
    W = tf.Variable(tf.zeros([784, 10]))

In [8]:
with tf.name_scope('biases'):
    b = tf.Variable(tf.zeros([10]))

In [9]:
with tf.name_scope('softmax'):
    y = tf.nn.softmax(tf.matmul(x, W) + b)

In [10]:
with tf.name_scope('cross_entropy'):
    cross_entropy = tf.reduce_mean(-tf.reduce_sum(y_ * tf.log(y), reduction_indices=[1]))

In [11]:
with tf.name_scope('train'):
    train_op = tf.train.GradientDescentOptimizer(learning_rate).minimize(cross_entropy)

In [12]:
with tf.name_scope('evaluation'):
    correct_prediction = tf.equal(tf.argmax(y, 1), tf.argmax(y_, 1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))

In [13]:
tf.summary.scalar('cost', cross_entropy)
tf.summary.scalar('accuracy', accuracy)

summary_op = tf.summary.merge_all()

In [20]:
with tf.Session() as sess:
    sess.run(tf.global_variables_initializer())
    writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    for epoch in range(training_epochs):
        batch_count = int(mnist.train.num_examples / batch_size)
        for i in range(batch_count):
            batch_x, batch_y = mnist.train.next_batch(batch_size)
            _, summary = sess.run([train_op, summary_op], feed_dict={x : batch_x, y_ : batch_y})
            writer.add_summary(summary, epoch * batch_count + i)
        
        if epoch % 5 == 0: 
            print("Epoch: ", epoch)
            print("\tAccuracy: ", accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))
            print("\tCost: ", cross_entropy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

    print("done")

Epoch:  0
	Accuracy:  0.9011
	Cost:  0.367447
Epoch:  5
	Accuracy:  0.9172
	Cost:  0.289219
Epoch:  10
	Accuracy:  0.9223
	Cost:  0.277612
Epoch:  15
	Accuracy:  0.921
	Cost:  0.273392
Epoch:  20
	Accuracy:  0.9226
	Cost:  0.271672
Epoch:  25
	Accuracy:  0.9238
	Cost:  0.269809
Epoch:  30
	Accuracy:  0.9217
	Cost:  0.270563
Epoch:  35
	Accuracy:  0.9236
	Cost:  0.268846
Epoch:  40
	Accuracy:  0.9239
	Cost:  0.266768
Epoch:  45
	Accuracy:  0.9246
	Cost:  0.266457
done
